# Loss of "state IV": $\lvert\psi\rangle_\text{in} = N \left( \hat{a}_1^\dagger \lvert\alpha_1,\alpha_2\rangle \right) + \left( \hat{a}_2^\dagger \lvert\alpha_1,\alpha_2\rangle \right)$

In [1]:
%matplotlib inline

from math import *
from qutip import *
from pylab import *

from scipy.optimize import minimize as cp_minimize

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### Symbols & operators

In [2]:
T = sp.symbols('T')
N = 10 # dimension of the Hilbert space

# operators
I = qeye(N) # identity operator
a = destroy(N) # annihilation operator
n = num(N); n_a = tensor(n,I,I,I); n_b = tensor(I,n,I,I) # number operators
D_n = n_a - n_b # ΔN = n_1 - n_2
S_n = n_a + n_b # ΣN = n_1 + n_2

# Schwinger representation
Jx = 1/2*(tensor(a.dag(),a) + tensor(a,a.dag()))
Jy = -1j/2*(tensor(a.dag(),a) - tensor(a,a.dag()))
Jz = 1/2*(tensor(a.dag()*a,I) - tensor(I,a.dag()*a))

# Parity operator
Pa = (1j*np.pi*tensor(a.dag()*a,I,I,I)).expm()
Pb = (1j*np.pi*tensor(I,a.dag()*a,I,I)).expm()
P = (1j*np.pi*a.dag()*a).expm()

### Functions for input states

In [3]:
# returns ρ_in given pure state ψ_in
def rho(psi):
    return psi*psi.dag()

# |ψ_in_4> = N(a_1†|α_1,α_2>+a_2†|α_1,α_2>)
def psi_in_4(alpha_1, alpha_2):
    psi_alphas = tensor(coherent(N, alpha_1),coherent(N, alpha_2)) # |α_1,α_2>
    temp = tensor(a.dag(), I)*psi_alphas + tensor(I, a.dag())*psi_alphas
    return temp.unit()

## Numerical Heisenberg picture 2

In [4]:
def phi_std_dev(rho_in,rho_cd,L):
    # ΔP
    Pa_std_dev = np.sqrt(1 - (tensor(rho_in,rho_cd)*Pa).tr()**2)
    
    # |d<P_a>_out/dφ|
    result = []
    for phi in arange(-0.01,2*np.pi,0.01):
        # MZI transformation operator & derivative
        M = (-1j*Jy*phi).expm();
        dM = (-1j*Jy)*M;
        
        # derivative: rho leaving lossless MZI
        drho_ab  = dM*rho_in*M.dag() + M*rho_in*dM.dag()
        # derivative: rho leaving "loss splitters"
        drho_out = L*tensor(drho_ab,rho_cd)*L.dag()
        
        value = np.abs((drho_out*Pa).tr())
        print(value)
        result.append(value)
        
    return(Pa_std_dev/np.array(result))

#### Caculating $\Delta\phi_\text{min}$ vs. $\langle n \rangle$ using $\hat{P} = (-1)^{\hat{N_1}}$

In [5]:
file = open("parity_numerical_Heisenberg_2_loss_4.txt","w+")
file.write("# Δφ_min calculated using parity operator and numerical Heisenberg method with loss\n")
file.write("# |ψ_in_4> = N(a_1†|α_1,α_2>+a_2†|α_1,α_2>)\n")
file.write("# ------------------------------------------------------------------------------------------------\n")
file.write("# %-10s %-11s %-30s\n" % ("alpha_1","alpha_2","Δφ_min_4"))

56

In [6]:
# "loss operator"
l = 0.01; theta = np.arcsin(np.sqrt(1-l))
L = (1j*theta*(tensor(a.dag(),I,a,I)+tensor(a,I,a.dag(),I)+tensor(I,a.dag(),I,a)+tensor(I,a,I,a.dag()))).expm()

# vaccum state |00>_c,d
rho_cd = rho(tensor(fock(N,0),fock(N,0)))

In [7]:
n_4_avgs = []; phi_4_std_dev_mins = []

alpha_1s = [0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25]
alpha_2s = [-0.25, -0.5, -0.75, -1, -1.25, -1.5, -1.75, -2, -2.25]
for alpha_1,alpha_2 in zip(alpha_1s,alpha_2s):
    rho_in_4 = rho(psi_in_4(alpha_1,alpha_2))
    
    # <n>'s
    n_4_avgs.append(np.abs((tensor(rho_in_4,rho_cd)*S_n).tr()))

    # Δφ_min's
    phi_4_std_dev = phi_std_dev(rho_in_4,rho_cd,L)
    min_4 = np.amin(phi_4_std_dev)
    phi_4_std_dev_mins.append(min_4)
    
    file.write("  %-10.8f %-10.8f %-30.25f\n" % (alpha_1,alpha_2,min_4))
    print("alpha_1 =", alpha_1, ", alpha2_=", alpha_2)

0.04379589121587972
0.04378602722261221
0.04377178638879689
0.043753173749159456
0.043730194785413706
0.04370285539249274
0.04367116190676989
0.0436351210937401
0.04359474014455977
0.043550026668026744
0.04350098870082133
0.0434476346964165
0.043389973529264846
0.04332801448345412
0.04326176725699378
0.04319124196183264
0.04311644910856699
0.043037399625731776
0.04295410483477014
0.042866576459116915
0.042774826628743184
0.042678867855185584
0.04257871305533509
0.04247437552736012
0.0423658689600047
0.04225320743318725
0.04213640539181754
0.04201547767804958
0.04189043951003718
0.04176130647020408
0.041628094506741986
0.041490819955153586
0.04134949950717753
0.041204150216044275
0.041054789491632195
0.040901435112465
0.040744105202402736
0.04058281823624191
0.040417593039601504
0.040248448787570834
0.04007540498918588
0.039898481497324174
0.03971769850246623
0.03953307653141435
0.03934463642964303
0.03915239937943696
0.03895638688997311
0.03875662078714035
0.038553123216309826
0.038345

In [8]:
file.close()